In [1]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import EAtools as ea
import pandas as pd
import os   
import sys 
import numpy
import fileinput
import matplotlib.pyplot as plt 
import matplotlib.ticker  
from matplotlib.ticker import FormatStrFormatter 
import pandas.io.sql
import pyodbc 
from matplotlib.pyplot import figure, show
from matplotlib.patches import Ellipse
import numpy as np
from pandas import *
import EAtools as ea
ea.set_options()
ea.ea_report_style()
from pylab import *
import matplotlib.ticker as tkr
from datetime import datetime, date, time, timedelta
ea_p=ea.ea_p
ea_s=ea.ea_s
def cm2inch(value):
    return value/2.54
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
from db import DB
import os
ea.set_options()
ea.ea_report_style()
%pylab inline
%matplotlib inline
path='/media/usb/notebooks/Julia/20192020_review/'
db = DB(profile="hallj") 
#db = DB(profile="millerr") 

Populating the interactive namespace from numpy and matplotlib


/home/julia/.local/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime', 'fft', 'info', 'linalg', 'random', 'power']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# create parquet file with new point of connection

In [5]:
Q=r"""

SELECT

	   [Participant]
      ,[Tranche]
      ,[TradingDate]
      ,[TradingPeriod]
      ,[PointOfConnection]
      ,[Unit]
      ,[Megawatt]
      ,[DollarsPerMegawattHour]

  FROM [ElectricityAuthorityData].[wholesale].[Offers]

  where PointOfConnection in ('ARI1101','ARI1102','ARA2201','ATI2201','KPO1101','MTI2201','OHK2201','WPA2201','WKM2201',
                         'BEN2202','AVI2201','OHA2201','OHB2201','OHC2201','WTK0111','CYD2201','MAN2201','ROX1101','ROX2201',
                         'TKA0111','TKB2201','TUI1101', 'RPO2201', 'TKU2201',
                         'HLY2201','SFD2201','MKE1101','WHI2201','JRD1101')
  and ProductType='Energy' and IsLatest='Y' and  
  TradingDate>='2014-1-1' and TradingDate<='2021-6-30'
  order by TradingDate, TradingPeriod,[PointOfConnection],[Unit], Tranche

"""
##[Participant]in ('MERI', 'MEEN', 'MRPL', 'CTCT')
 
Offers= db.query(Q)
Offers.tail()

,Participant,Tranche,TradingDate,TradingPeriod,PointOfConnection,Unit,Megawatt,DollarsPerMegawattHour
24205930,MERI,1,2021-06-30,48,WTK0111,WTK0,47.0,0.01
24205931,MERI,2,2021-06-30,48,WTK0111,WTK0,0.0,135.00
24205932,MERI,3,2021-06-30,48,WTK0111,WTK0,0.0,250.00
24205933,MERI,4,2021-06-30,48,WTK0111,WTK0,0.0,350.00
24205934,MERI,5,2021-06-30,48,WTK0111,WTK0,28.0,485.00


In [6]:
Offers['TradingDate']=pandas.to_datetime(Offers['TradingDate'], dayfirst=True)
offers=Offers.copy()
offers.head()

,Participant,Tranche,TradingDate,TradingPeriod,PointOfConnection,Unit,Megawatt,DollarsPerMegawattHour
0,MRPL,1,2014-01-01,1,ARA2201,ARA0,22.0,0.0
1,MRPL,2,2014-01-01,1,ARA2201,ARA0,0.0,22.0
2,MRPL,3,2014-01-01,1,ARA2201,ARA0,0.0,41.0
3,MRPL,4,2014-01-01,1,ARA2201,ARA0,4.0,94.0
4,MRPL,5,2014-01-01,1,ARA2201,ARA0,52.0,165.0


In [7]:
#make a new column that is the point of connection for Clyde and Manapouri, is equal to 'Roxburgh' for both Rox nodes,
#is equal to 'Waitaki' for BEN, AVI, OHA, OHB, OHC, WTK
#and is equal to 'Waikato' for the Waitako nodes
#ie - treating the nodes where there is block dispatch as one station

def newptofconnection(row):
    if row['PointOfConnection'] in ('MAN2201'):
        return 'MAN'
    if row['PointOfConnection'] in ('BEN2202','AVI2201','OHA2201','OHB2201','OHC2201','WTK0111'):
        return 'WAITAKI'
    if row['PointOfConnection'] in ('CYD2201'):
        return 'CYD'
    if row['PointOfConnection'] in ('ROX1101','ROX2201'):
        return 'ROX'
    if row['PointOfConnection'] in ('TKA0111','TKB2201'):
         return 'TKA'
    if (row['PointOfConnection'] in ('ARI1101','ARI1102','ARA2201','ATI2201','KPO1101','MTI2201','OHK2201','WPA2201','WKM2201')) & (row['Participant']=='MRPL'):
        return 'TAUPO'
    if row['PointOfConnection'] in ('TUI1101'):
        return 'WAIKAREMOANA'
    if row['PointOfConnection'] in ('RPO2201','TKU2201'):
        return 'TONGARIRO'
    if (row['PointOfConnection'] in ('HLY2201')) & (row['Unit'] in ('HLY1','HLY2','HLY3','HLY4')):
        return 'RANKINES'
    if (row['PointOfConnection'] in ('HLY2201')) & (row['Unit'] in ('HLY5')):
        return 'E3P'
    if (row['PointOfConnection'] in ('HLY2201')) & (row['Unit'] in ('HLY6')):
        return 'HLY6'
    if (row['PointOfConnection'] in ('SFD2201')) & (row['Unit'] in ('SPL0')):
        return 'TCC'
    if (row['PointOfConnection'] in ('SFD2201')) & (row['Unit'] in ('SFD21','SFD22')):
        return 'PEAKERS'
    if row['PointOfConnection'] in ('MKE1101'):
        return 'MKE' 
    if row['PointOfConnection'] in ('WHI2201'):
        return 'WHI' 
    if row['PointOfConnection'] in ('JRD1101'):
        return 'JRD' 
    
offers['NewPointOfConnection'] = offers.apply(newptofconnection,axis=1)
offers.head()

,Participant,Tranche,TradingDate,TradingPeriod,PointOfConnection,Unit,Megawatt,DollarsPerMegawattHour,NewPointOfConnection
0,MRPL,1,2014-01-01,1,ARA2201,ARA0,22.0,0.0,TAUPO
1,MRPL,2,2014-01-01,1,ARA2201,ARA0,0.0,22.0,TAUPO
2,MRPL,3,2014-01-01,1,ARA2201,ARA0,0.0,41.0,TAUPO
3,MRPL,4,2014-01-01,1,ARA2201,ARA0,4.0,94.0,TAUPO
4,MRPL,5,2014-01-01,1,ARA2201,ARA0,52.0,165.0,TAUPO


In [8]:
def newptofconnection2(row):
    if row['PointOfConnection'] in ('MAN2201'):
        return 'MAN'
    if row['PointOfConnection'] in ('BEN2202','AVI2201','OHA2201','OHB2201','OHC2201','WTK0111'):
        return 'WAITAKI'
    if row['PointOfConnection'] in ('CYD2201','ROX1101','ROX2201'):
        return 'CLUTHA'
    if row['PointOfConnection'] in ('TKA0111','TKB2201'):
         return 'TKA'
    if (row['PointOfConnection'] in ('ARI1101','ARI1102','ARA2201','ATI2201','KPO1101','MTI2201','OHK2201','WPA2201','WKM2201')) & (row['Participant']=='MRPL'):
        return 'TAUPO'
    if row['PointOfConnection'] in ('TUI1101'):
        return 'WAIKAREMOANA'
    if row['PointOfConnection'] in ('RPO2201','TKU2201'):
        return 'TONGARIRO'
    if (row['PointOfConnection'] in ('HLY2201')):
        return 'HUNTLY'
    if (row['PointOfConnection'] in ('SFD2201')):
        return 'STRATFORD'
    if row['PointOfConnection'] in ('MKE1101'):
        return 'MKE' 
    if row['PointOfConnection'] in ('WHI2201'):
        return 'WHI' 
    if row['PointOfConnection'] in ('JRD1101'):
        return 'JRD' 
    
offers['NewPointOfConnection2'] = offers.apply(newptofconnection2,axis=1)
offers.head()

,Participant,Tranche,TradingDate,TradingPeriod,PointOfConnection,Unit,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2
0,MRPL,1,2014-01-01,1,ARA2201,ARA0,22.0,0.0,TAUPO,TAUPO
1,MRPL,2,2014-01-01,1,ARA2201,ARA0,0.0,22.0,TAUPO,TAUPO
2,MRPL,3,2014-01-01,1,ARA2201,ARA0,0.0,41.0,TAUPO,TAUPO
3,MRPL,4,2014-01-01,1,ARA2201,ARA0,4.0,94.0,TAUPO,TAUPO
4,MRPL,5,2014-01-01,1,ARA2201,ARA0,52.0,165.0,TAUPO,TAUPO


In [9]:
offers.to_csv(path+'/offers_NewPointOfConnectoin.csv')